In [14]:
%run SetUpParams.ipynb

2024-08-13 01:21:34.138960
2004-08-13 01:21:34.138960


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [15]:
import pandas as pd 
import numpy as np

### Create dataframe to feed into model

In [16]:
final_df = pd.DataFrame()
# Next Day Open Price - Current Day Open Price
final_df[f'{target_stock}_change'] = target_stock_data['Open'].shift(-1) - target_stock_data['Open']
final_df[f'{target_stock}_lag'] = final_df[f'{target_stock}_change'].shift(1)
final_df.reset_index(inplace=True)
print(final_df)

           Date  ^GSPC_change  ^GSPC_lag
0    2004-08-13      1.570068        NaN
1    2004-08-16     14.539917   1.570068
2    2004-08-17      2.369995  14.539917
3    2004-08-18     13.460083   2.369995
4    2004-08-19     -3.940063  13.460083
...         ...           ...        ...
5028 2024-08-06     86.709961  55.279785
5029 2024-08-07    -40.560059  86.709961
5030 2024-08-08     62.090332 -40.560059
5031 2024-08-09     37.219727  62.090332
5032 2024-08-12           NaN  37.219727

[5033 rows x 3 columns]


#### Add European and US Stock Market Price Changes to final_df
Current Day Open Price - Previous Day Open Price

In [17]:
# EU Stock Market has already opened
for ticker in US_EU_market_dict.keys():
        data = US_EU_market_dict[ticker]
        # Current Day Open Price - Previous Day Open Price
        temp = (data['Open'] - data['Open'].shift(1)).reset_index()
        temp.rename(columns={'Open':ticker}, inplace=True)
        final_df = final_df.merge(temp, on='Date', how='left')

print(final_df)

           Date  ^GSPC_change  ^GSPC_lag       ^IXIC        ^DJI      ^FCHI  \
0    2004-08-13      1.570068        NaN         NaN         NaN        NaN   
1    2004-08-16     14.539917   1.570068   -2.880005   11.239258   3.310059   
2    2004-08-17      2.369995  14.539917   32.680054  130.150391  42.409912   
3    2004-08-18     13.460083   2.369995   -4.599976    8.719727  15.120117   
4    2004-08-19     -3.940063  13.460083   38.250000  118.560547  29.449951   
...         ...           ...        ...         ...         ...        ...   
5028 2024-08-06     86.709961  55.279785  548.830078 -319.972656  60.479980   
5029 2024-08-07    -40.560059  86.709961  360.950195  493.871094   1.870117   
5030 2024-08-08     62.090332 -40.560059 -214.041016 -289.710938  61.069824   
5031 2024-08-09     37.219727  62.090332  228.250000  467.679688  36.690430   
5032 2024-08-12           NaN  37.219727   -0.019531  147.941406  37.139648   

          ^GDAXI  
0            NaN  
1      -5.809

#### Add Asian Stock Market Price Changes to final_df
Close Price - Open Price

In [18]:
# Asian Stock Market has already closed; one day ahead of US Stock Market
for ticker in asia_market_dict.keys():
        data = asia_market_dict[ticker]
        # Close Price - Open Price
        temp = (data['Close'] - data['Open']).reset_index()
        temp.rename(columns={0:ticker}, inplace=True)
        final_df = final_df.merge(temp, on='Date', how='left')

print(final_df)

           Date  ^GSPC_change  ^GSPC_lag       ^IXIC        ^DJI      ^FCHI  \
0    2004-08-13      1.570068        NaN         NaN         NaN        NaN   
1    2004-08-16     14.539917   1.570068   -2.880005   11.239258   3.310059   
2    2004-08-17      2.369995  14.539917   32.680054  130.150391  42.409912   
3    2004-08-18     13.460083   2.369995   -4.599976    8.719727  15.120117   
4    2004-08-19     -3.940063  13.460083   38.250000  118.560547  29.449951   
...         ...           ...        ...         ...         ...        ...   
5028 2024-08-06     86.709961  55.279785  548.830078 -319.972656  60.479980   
5029 2024-08-07    -40.560059  86.709961  360.950195  493.871094   1.870117   
5030 2024-08-08     62.090332 -40.560059 -214.041016 -289.710938  61.069824   
5031 2024-08-09     37.219727  62.090332  228.250000  467.679688  36.690430   
5032 2024-08-12           NaN  37.219727   -0.019531  147.941406  37.139648   

          ^GDAXI       ^AORD        ^HSI        ^N2

In [19]:
# Different markets has different holidays
# Forward fill null values
final_df.ffill(inplace=True)
final_df.dropna(inplace=True)
# final_df.isna().sum()
final_df

,Date,^GSPC_change,^GSPC_lag,^IXIC,^DJI,^FCHI,^GDAXI,^AORD,^HSI,^N225,^NSEI
778,2007-09-17,-7.609985,0.290039,11.890137,20.560547,-16.479980,-17.530273,-23.299805,-378.080078,232.370117,-23.800293
779,2007-09-18,43.119995,-7.609985,-1.080078,-38.770508,-104.929688,-37.049805,-66.500000,73.019531,-235.690430,52.100098
780,2007-09-19,8.939941,43.119995,72.929932,337.430664,264.319824,169.069824,113.100098,43.281250,343.419922,182.100098
781,2007-09-20,-9.939941,8.939941,-6.309814,72.909180,2.810059,99.669922,14.299805,62.820312,-60.871094,12.699707
782,2007-09-21,7.000000,-9.939941,11.169922,-45.189453,-18.060059,-5.809570,-22.099609,183.818359,28.180664,84.599609
...,...,...,...,...,...,...,...,...,...,...,...
5028,2024-08-06,86.709961,55.279785,548.830078,-319.972656,60.479980,-103.019531,30.700195,-213.960938,2598.130859,-197.298828
5029,2024-08-07,-40.560059,86.709961,360.950195,493.871094,1.870117,16.080078,23.000000,193.080078,967.269531,8.099609
5030,2024-08-08,62.090332,-40.560059,-214.041016,-289.710938,61.069824,77.400391,-26.600098,118.791016,185.359375,-131.550781
5031,2024-08-09,37.219727,62.090332,228.250000,467.679688,36.690430,156.849609,104.200195,-7.128906,-247.339844,-19.349609
